DATA: https://www.kaggle.com/datasets/declanmcalinden/time-series-uk-supermarket-data

In [2]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
DATA_PATH = "C:/Project/UK store analysis/data/00_raw/"
INTERIM_PATH = "C:/Project/UK store analysis/data/01_interim/"

In [3]:
all_files = glob.glob(os.path.join(DATA_PATH, "All_Data_*.csv"))
print(all_files)

['C:/Project/UK store analysis/data/00_raw\\All_Data_Aldi.csv', 'C:/Project/UK store analysis/data/00_raw\\All_Data_ASDA.csv', 'C:/Project/UK store analysis/data/00_raw\\All_Data_Morrisons.csv', 'C:/Project/UK store analysis/data/00_raw\\All_Data_Sains.csv', 'C:/Project/UK store analysis/data/00_raw\\All_Data_Tesco.csv']


In [4]:
# Load concatenate Data
all_files = glob.glob(os.path.join(DATA_PATH, "All_Data_*.csv"))

# Specify dtypes to handle the ASDA warning proactively
# Handle own_brand column properly later
dtype_spec = {
    "supermarket": "category",
    'prices_(£)': 'float64',
    'prices_unit_(£)': 'float64',
    'unit': 'object',
    'names': 'object',
    'date': 'str', # Load as string to ensure correct parsing
    'category': 'category', # More memory efficient
    'own_brand': 'object'
}
df_list = []
for f in all_files:
    print(f"Loading {f}...")
    df_temp = pd.read_csv(f, dtype=dtype_spec)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True)

print("\nData Loading and concatenated successfully")
df.info()

Loading C:/Project/UK store analysis/data/00_raw\All_Data_Aldi.csv...
Loading C:/Project/UK store analysis/data/00_raw\All_Data_ASDA.csv...
Loading C:/Project/UK store analysis/data/00_raw\All_Data_Morrisons.csv...
Loading C:/Project/UK store analysis/data/00_raw\All_Data_Sains.csv...
Loading C:/Project/UK store analysis/data/00_raw\All_Data_Tesco.csv...

Data Loading and concatenated successfully
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9529242 entries, 0 to 9529241
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   supermarket      object 
 1   prices_(£)       float64
 2   prices_unit_(£)  float64
 3   unit             object 
 4   names            object 
 5   date             object 
 6   category         object 
 7   own_brand        object 
dtypes: float64(2), object(6)
memory usage: 581.6+ MB


In [11]:
# Clean column names by removing special chars and standardise case
import re

df.columns = (
    df.columns
    .str.lower()
    .str.replace(r"[^\w]+", "_", regex=True)
    .str.strip("_")
)

print("Cleaned column names:")
print(df.columns)


Cleaned column names:
Index(['supermarket', 'prices', 'prices_unit', 'unit', 'names', 'date',
       'category', 'own_brand'],
      dtype='object')


In [12]:
# 2. Correct Data Types
# Convert date from integer to proper datetime
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

In [13]:
# 3. Standardise own brand column
print("\nUnique values in 'own_brand' before standardization:")
print(df["own_brand"].value_counts(dropna=False))


Unique values in 'own_brand' before standardization:
own_brand
False    7131233
True     2398009
Name: count, dtype: int64


In [14]:
mapping = {True: True, "True": True, False: False, "False": False}
df["own_brand"] = df["own_brand"].map(mapping).fillna(False).astype(bool)

print("\nUnique values in 'own_brand' after standardization:")
print(df["own_brand"].value_counts(dropna=False))



Unique values in 'own_brand' after standardization:
own_brand
False    7131233
True     2398009
Name: count, dtype: int64


In [15]:
# 4. Basic string cleaning
# Standardise text columns for easier matching later
for col in ["names", "category"]:
    df[col] = df[col].str.strip()

print("Data types after initial cleaning")
df.info()

Data types after initial cleaning
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9529242 entries, 0 to 9529241
Data columns (total 8 columns):
 #   Column       Dtype         
---  ------       -----         
 0   supermarket  object        
 1   prices       float64       
 2   prices_unit  float64       
 3   unit         object        
 4   names        object        
 5   date         datetime64[ns]
 6   category     object        
 7   own_brand    bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(4)
memory usage: 518.0+ MB


## Data Quality Deep Dive and Issue Logging

### Investigating Outrageous price_unit values

In [20]:
print("\nInvestigating extreme prices_unit value:")
print(df.describe()["prices_unit"])



Investigating extreme price_unit value:
count    9.528718e+06
mean     4.609825e+01
min      0.000000e+00
25%      3.440000e+00
50%      7.700000e+00
75%      1.500000e+01
max      1.500000e+05
std      6.506242e+02
Name: prices_unit, dtype: float64

Top 10 most expensive items by unit price: 
        supermarket                                   names  prices  \
8336288       Tesco   Lancashire Farm Whole Milk Yogurt 1Kg  150.00   
8504816       Tesco   Lancashire Farm Whole Milk Yogurt 1Kg  150.00   
7835671       Tesco  Scholl Fungal Nail Treatment Foot Care  421.05   
471499         ASDA         Max Factor Brow Shaper 20 Brown    8.98   
471501         ASDA    Max Factor Brow Shaper 30 Deep Brown    8.98   
471526         ASDA        Max Factor Brow Shaper 10 Blonde    8.98   
516849         ASDA    Max Factor Brow Shaper 30 Deep Brown    8.98   
516858         ASDA         Max Factor Brow Shaper 20 Brown    8.98   
516878         ASDA        Max Factor Brow Shaper 10 Blonde    8.

In [22]:
print("Top 10 most expensive items by unit price: ")
df.nlargest(10, "prices_unit")[["supermarket", "names", "prices", "prices_unit", "unit"]]


Top 10 most expensive items by unit price: 


,supermarket,names,prices,prices_unit,unit
8336288,Tesco,Lancashire Farm Whole Milk Yogurt 1Kg,150.00,150000.0,kg
8504816,Tesco,Lancashire Farm Whole Milk Yogurt 1Kg,150.00,150000.0,kg
7835671,Tesco,Scholl Fungal Nail Treatment Foot Care,421.05,110803.5,l
471499,ASDA,Max Factor Brow Shaper 20 Brown,8.98,99778.0,kg
471501,ASDA,Max Factor Brow Shaper 30 Deep Brown,8.98,99778.0,kg
471526,ASDA,Max Factor Brow Shaper 10 Blonde,8.98,99778.0,kg
516849,ASDA,Max Factor Brow Shaper 30 Deep Brown,8.98,99778.0,kg
516858,ASDA,Max Factor Brow Shaper 20 Brown,8.98,99778.0,kg
516878,ASDA,Max Factor Brow Shaper 10 Blonde,8.98,99778.0,kg
538380,ASDA,Max Factor Brow Shaper 30 Deep Brown,8.98,99778.0,kg


Hypothesis: Is this a 'price per 100g' vs 'price per kg' issue.
For example, if an item is £10 for 100g, its price per kg would be £100.
If it's £997.78 for 100g, its price per kg would be £99778.
This looks like a unit conversion error during data scraping/generation.

Solutions:
1. Correct them if a clear pattern emerges.
2. Remove them if they are clear, uncorrectable errors.

### B. Investigating Zero and Missing Prices

In [26]:
print(f"Number of items with price = 0: {len(df[df["prices"] == 0])}")
print("Examples of items with price = 0:")
print(df[df["prices"] == 0].head())
# Hypothesis: These could be data errors or "buy one get one free" type promotions where the free item is listed at 0.

Number of items with price = 0: 3
Examples of items with price = 0:
        supermarket  prices  prices_unit unit  \
7789212       Tesco     0.0          NaN  NaN   
8086680       Tesco     0.0          0.0   kg   
8854571       Tesco     0.0          NaN  NaN   

                                                names       date  \
7789212                       Tesco Peppered Ham 120g 2024-03-25   
8086680  Tahira Chicken Smoked Sausages Hot Dogs 500G 2024-03-11   
8854571  Twix Multipack Chocolate Bars Biscuits 9x20g 2024-02-08   

              category  own_brand  
7789212     fresh_food       True  
8086680  food_cupboard      False  
8854571  food_cupboard      False  


In [30]:
# Investigating missng/zero price_unit
print(f"Number of items with prices_unit = 0: {len(df[df["prices_unit"] == 0])}")
print(f"Number of items with missing prices_unit: {df["prices_unit"].isnull().sum()}")

print("\nUnits for items with prices_unit = 0:")
print(df[df["prices_unit"] == 0]["unit"].value_counts().head())

Number of items with prices_unit = 0: 12596
Number of items with missing prices_unit: 524

Units for items with prices_unit = 0:
unit
unit    10565
kg       1687
m         179
l         165
Name: count, dtype: int64


### C. Product Name Standardisation

In [33]:
print(f"Total unique product names: {df["names"].nunique()}")

# Look at some examples of common product
print("\nExamples of 'milk' product names: ")
print(df[df["names"].str.contains("milk", case=False, na=False)]["names"].sample(10).tolist())

Total unique product names: 127226

Examples of 'milk' product names: 
['Kingfisher Oriental Coconut Milk Rich & Creamy 200Ml', 'Cravendale Filtered Fresh Whole Milk Fresher for Longer', 'Nesquik Banana Flavour Milkshake Mix', 'Tesco Milk Chocolate Turkish Delight Thins 180G', 'Milky Way Dairy Free Magic Stars 25g', 'Milkybar Milk Slice', 'Mars Chocolate Milk Shake Drink No Added Sugar', 'Cravendale Filtered Fresh Semi Skimmed Milk Fresher for Longer 3L', 'Nesquik Chocolate Milkshake Powder Tub 500g', 'Cadbury dairy milk chocolate buttons twisted']


In [34]:
# Save the cleaned dataset
os.makedirs(INTERIM_PATH, exist_ok=True)

file_path = os.path.join(INTERIM_PATH, "cleaned_supermarket_data.parquet")
df.to_parquet(file_path)

print(f"Cleaned data saved to {file_path}")

Cleaned data saved to C:/Project/UK store analysis/data/01_interim/cleaned_supermarket_data.parquet


1. The prices_unit Outliers are Real (Mostly!): nlargest output is a goldmine.

* Scholl Fungal Nail Treatment & Max Factor Brow Shaper: The extremely high prices_unit are due to very expensive products sold in tiny quantities (milliliters or grams). A price of £8.98 for a 0.09g brow shaper results in a per-kg price of ~£99,778. This is correct math, not bad data. It highlights an important feature of the dataset: we must be careful when comparing unit prices across categories (e.g., cosmetics vs. potatoes).

* Lancashire Farm Yogurt: This is the real data error. A 1Kg yogurt would not cost £150.00. It's overwhelmingly likely that the price should be £1.50 and was entered incorrectly (perhaps in pence, or a misplaced decimal). This single finding is a fantastic demonstration of data validation. 

2. Zero/Missing Prices are Understood:
* price = 0: Very rare (3 instances), likely data errors or specific promotions. Negligible impact.
* prices_unit = 0 or NaN: Mostly for items sold by 'unit' (each), which is logical.